# Vllm

<center>
<img src="https://docs.vllm.ai/en/stable/assets/logos/vllm-logo-text-light.png" alt="vllm logo" width="400"/>
</center>

Vllm is less cool and has an ugly logo :) 

### 🏳️ Open the cc-in2p3 Jupyter Notebook launcher at 

> `https://notebook.cc.in2p3.fr`


launch a new terminal session

<center>
    <img src="https://icon-library.com/images/bash-icon/bash-icon-6.jpg" alt="bash is also cool" width="200"/>
</center>


and execute therein the following commands to load the previously prepared enviroment with ollama installed

```bash
$ module load Programming_Languages/python/3.11.4
$ source /pbs/throng/training/universite-hiver/wschooloptim/vllm/bin/activate
```

launch an vllm server

```bash
$ vllm serve HuggingFaceH4/zephyr-7b-beta --max_model_len=10000 --guided_decoding_backend=xgrammar --seed=1  --tensor-parallel-size 1 &
```

Create one request to ollama's server:

```bash
$ curl -X POST http://localhost:8000/v1 \
-H "Content-Type: application/json" \
-d '{
  "model": "zephyr:7b",
  "messages": [{ "role": "user", "content": "Who won the last presidential elections in France?" }]
}'
```

Keep that terminal open for the rest of the tutorial, we will check therein ollama server logs.

### 🏴 Open another cc-in2p3 Jupyter Notebook launcher at 

> `https://notebook.cc.in2p3.fr`


and launch now the Notebook called `Python 3.13 (Ollama)`.

We will work from now in that notebook.

In [1]:
! /pbs/throng/training/universite-hiver/wschooloptim/vllm/bin/xan tail /pbs/home/u/univ-hiver01/private/resinwinterschool/tweets_spanish_english_25K.csv | /pbs/throng/training/universite-hiver/wschooloptim/vllm/bin/xan v 


Displaying 3 cols from 10 rows of <stdin>
┌───┬───────┬───────────────────┬──────────────────────────────────────────┐
│ - │ index │ text              │ english                                  │
├───┼───────┼───────────────────┼──────────────────────────────────────────┤
│ 0 │ 24990 │ RT @joseantoniok… │ RT @joseantoniokast: There is no time t… │
│ 1 │ 24991 │ RT @reloaled: Pe… │ But look, the Minister is walking with … │
│ 2 │ 24992 │ RT @25Hernan25: … │ RT @25Hernan25: @verge_cdmx is the new … │
│ 3 │ 24993 │ RT @Sinfiltros_t… │ RT @Sinfiltros_tv: "Chilenians want a f… │
│ 4 │ 24994 │ RT @hernan_sr: I… │ RT @hernan_sr: It's amazing to see a yo… │
│ 5 │ 24995 │ RT @GAMBA_CL: CA… │ RT @GAMBA_CL: CADEM:Chilians see Kast a… │
│ 6 │ 24996 │ RT @delosquesobr… │ RT @delosquesobran: 🔴Daniel Stingo and… │
│ 7 │ 24997 │ RT @Daniel399492… │ RT @Daniel399492892: Bernardo Fontaine,… │
│ 8 │ 24998 │ RT @joseantoniok… │ RT @joseantoniokast: There's no time to… │
│ 9 │ 24999 │ RT @csantander23… │ 

##### Import python packages

In [2]:
import csv
import time
import asyncio
from openai import OpenAI
from string import Template
import nest_asyncio
nest_asyncio.apply()

##### Load inputs and create asynchronous iterator

In [3]:
input_file = "/pbs/home/u/univ-hiver01/private/resinwinterschool/tweets_spanish_english_25K.csv"
content_column = 'english'

start = time.time()
with open(input_file, 'r') as f:
    tweets = [d[content_column] for d in csv.DictReader(f)]
end = time.time()
print(f"Loading {len(tweets)} tweets tooks {end - start} seconds.")


Loading 25000 tweets tooks 0.1264493465423584 seconds.


##### Load choices and extra body dictionary with structured_outputs options

In [4]:
choices_file = "/pbs/throng/training/universite-hiver/wschooloptim/resinwinterschool/multiple.txt"
with open(choices_file, 'r') as f:
    choice = f.read().split("\n")

extra_body = {"structured_outputs": {"choice": choice}}

print(extra_body)

{'structured_outputs': {'choice': ['Jara', 'Kast', 'Kaiser', 'Matthei', 'Parisi', 'Artés', 'Mayne-Nichols', 'Enríquez-Ominami', 'None']}}


##### Load instructions

In [5]:
instructions_file = "/pbs/throng/training/universite-hiver/wschooloptim/resinwinterschool/instructions.txt"
with open(instructions_file, 'r') as f:
    instructions = f.read()
print(instructions)

Please classify the following social media message (posted in the weeks leading up to the 2025 Chilean presidential election) according to whether it explicitly expresses the intention  of the author to vote for or calls for a vote for any of the candidates in that election: Jeannette Jara, José Antonio Kast, Johannes Kaiser, Evelyn Matthei, Franco Parisi, Eduardo Artés, Harold Mayne-Nichols, Marco Enríquez-Ominami (also known as MEO), or whether it expresses neither of these intentions. Your answer should be based solely on the information contained in the message. Do not confuse a simple mention of a candidate with an intention or call to vote for him. Do not assume that a message containing only positive opinions about a particular candidate explicitly expresses the intention to vote for that candidate. Do not assume that a message corresponding to the opinions or political positions of a particular candidate necessarily expresses the intention to vote for that candidate. Do not con

##### Create ollama openai client

In [6]:
client = OpenAI(
    base_url="http://localhost:8000/v1", # this is the same HOST given at ollama server launching
    api_key="EMPTY"  # required but unused
)

##### Create async functions to asynchronously request llm server


In [7]:
async def inputsIterator(start, end):
    for tweet in tweets[start:end]:
        yield tweet

In [8]:
async def doCompletetion(tweet):
    return client.chat.completions.create(
        model=client.models.list().data[0].id,
        messages=[{
            'role': 'user',
            'content': Template(instructions).substitute(tweet=tweet)
        }],
        extra_body=extra_body,
        max_completion_tokens=16,
        temperature=0.7,
        top_p=0.95,
    )

In [9]:
async def run_all(start, end):
    # Asynchronously call the function for each prompt
    tasks = [
        doCompletetion(tweet)
        async for tweet in inputsIterator(start, end)
    ]
    # Gather and run the tasks concurrently
    results = await asyncio.gather(*tasks)
    return results

In [10]:
# Run all courutines
init_idx = 39
nb_tweets = 500

start = time.time()
results = asyncio.run(run_all(start=init_idx, end=init_idx + nb_tweets))
elapsed = time.time() - start

whole_db_nb_tweets = 1043873


print(f"""
Annotationg {nb_tweets} tweets took {elapsed} seconds,
this is {elapsed / nb_tweets} seconds per tweet or
{(1 / (24 * 3600)) * whole_db_nb_tweets * elapsed / nb_tweets} days for the whole database of {whole_db_nb_tweets} inputs.
""")


Annotationg 500 tweets took 27.607022047042847 seconds,
this is 0.055214044094085694 seconds per tweet or
0.6670885399377953 days for the whole database of 1043873 inputs.



> Annotationg 500 tweets took 27.607022047042847 seconds,
>
> tthis is 0.055214044094085694 seconds per tweet or
>
> 0.6670885399377953 days for the whole database of 1043873 inputs.

In [12]:
results[:3]

[ChatCompletion(id='chatcmpl-238e14f48abb41a68cd54bd5629ce286', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='None', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None, token_ids=None)], created=1765844228, model='HuggingFaceH4/zephyr-7b-beta', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=2, prompt_tokens=355, total_tokens=357, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None, prompt_token_ids=None, kv_transfer_params=None),
 ChatCompletion(id='chatcmpl-c9694ab6480f45b083ef6874d5b3cc6e', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='None', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None, token_ids=None)], 

In [25]:
for tweet, answer in zip(tweets, [r.choices[0].message.content for r in results]):
    if answer == "Jara":
        print("-----------------------------------------------" + "\n" + tweet + "\n" + answer)

-----------------------------------------------
RT @CNNChile: José Antonio Kast accuses Francisco Vidal of being "an activist for Jara" and that "he should have already left the position" at TVN due to alleged electoral interventionism.
https://t.co/5jPZYKDFAY
Jara
-----------------------------------------------
RT @lizibap: 📊 #Cadem confirms: @joseantoniokast not only leads in security, but also has the ability to:
-Grow the economy and generate employment (32%)
-Exercise authority and leadership (36%)
-Solve immigration problems (38%)
-Push for radical changes (36%) #LaFuerzaDelCambio https://t.co/t8Rj1ftBWs
Jara
-----------------------------------------------
RT @v8_falcon: What's the difference between Fontaine's blunder and Sanhueza's seriousness......it's not irony or joke to compare a 2nd or even 3rd division Kast team to the experience and seriousness of Matthei's 1st division team......👍🇨🇱👌
Jara
-----------------------------------------------
@ignaciobriones_ The UD is brillia